<a href="https://colab.research.google.com/github/dangthevang/DataVietNam/blob/master/ProcessScan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os
import time
import re
import numpy as np
import math

In [131]:
PATH_F0 = "/content/drive/MyDrive/Data Lake/Ingestion/Day 0/Scan/"
PATH_RAW_VIS='/content/drive/MyDrive/DataVIS/VietNam/Data Lake/Raw_VIS/'
PATH_F1 = f"{PATH_RAW_VIS}Day 0 /Scan/Financial/Financial_F1/"
PATH_F2 = f"{PATH_RAW_VIS}Day 0 /Scan/Financial/Financial_F2/"
PATH_F3 = f"{PATH_RAW_VIS}Day 0 /Scan/Financial/Financial_F3/"
PATH_RULE = f"{PATH_RAW_VIS}Data_Rule/"
LINK_QUARTER = "Quarter/"
LINK_YEAR = "Year/"

dict_path = {   "F0":{"Year": PATH_F0 + LINK_YEAR,
                    "Quarter": PATH_F0 + LINK_QUARTER,
                    },
                "F1":{"Year": PATH_F1 + LINK_YEAR,
                    "Quarter": PATH_F1 + LINK_QUARTER,
                    },
                "F2":{"Year": PATH_F2 + LINK_YEAR,
                    "Quarter": PATH_F2 + LINK_QUARTER,
                    },
                "F3":{"Year": PATH_F3 + LINK_YEAR,
                    "Quarter": PATH_F3 + LINK_QUARTER,
                    },
             }
#Lấy tất cả tên trường dữ liệu
file_balance = pd.read_excel(f'{PATH_RULE}Feature_Standard_Library.xlsx', sheet_name= 'Balance Sheet')
file_income = pd.read_excel(f'{PATH_RULE}Feature_Standard_Library.xlsx', sheet_name= 'Income Statement')
file_all = pd.concat([file_balance, file_income], ignore_index= True)

# F1

In [117]:


#đổi mã số bctc thành chuỗi
list_all_code = list(file_all['Ma_BCTC'])
list_str_all_code = []
for item in list_all_code:
    list_str_all_code.append(str(item))

#Tạo dict code_feature
dict_code = {}
for i in range(len(file_all)):
    dict_code[str(file_all['Ma_BCTC'][i])] = file_all['Feature'][i]

#CODE đọc và xử lí các tình huống của file

def proocess_file0(data):
    #data là dataframe đọc từ file scan, với header là None
    data_toang = data.copy()
    all_data = []
    for col in data_toang.columns:
        col_pro = []
        for item in list(data_toang[col]):
            list_split = str(item).split('\n')
            col_pro = col_pro + list_split
        all_data.append(col_pro)
    data_process = pd.DataFrame()
    for id in range(len(data_toang.columns)):
        try:
            data_process[data_toang.columns[id]] = all_data[id]
        except:
            pass
    return data_process

def process_file(data):
    #data là dataframe đọc từ file scan, với header là None
    data_toang = data.copy()
    list_split = []
    for item in data_toang[data_toang.columns[0]]:
        code = str(item).split(' ')[0]
        list_split.append(code)
    data_toang.insert(0, 'Code', list_split)
    return data_toang

def find_Ma_so(data_col):
    #data_col là list dữ liệu ở cột
    check = -1
    data_get = []
    count_true = 0
    count_false = 0
    for value in data_col:
        try:
            val = str(int(float(value)))
        except:
            try:
                val =  re.sub(value)
            except:
                val = value
                pass
        if val in list_str_all_code:
            count_true += 1
        else:
            count_false += 1

    if count_true/(len(data_col)-1) > 0.49:
        return True
    return False

def find_data_get(data_col):
    #data_col là list dữ liệu ở cột
    check = -1
    data_get = []
    count_true = 0
    count_false = 0
    for value in data_col:
        # print(value)
        # try:
        #     val = str(int(value))
        # except:
        val = str(value)
        if val == 'nan':
            data_get.append(-math.inf)
            count_true += 1
            continue
        elif val == '-':
            data_get.append(-math.inf)
            count_true += 1
            continue

        if '(' in str(val):
            val = val.replace('(', '-')
        # print(val)
        val_process = val.replace('.', '').replace(',', '').replace(';', '').replace('\t', '').replace('\n', '').replace(')', '').replace(' ', '')
        try:
            data_get.append(float(val_process))
            # print(val_process, val, value)
            count_true += 1   
        except:
            check = 0
            data_get.append(-math.inf)
            count_false += 1

    if (max(data_get) > 10000 or (max(data_get) < 1 and max(data_get) >= 0)) and count_true/len(data_col) > 0.4 and count_false/len(data_col) < 0.51:
        check = True
    else:
        check = False
    return check, data_get     

def formatTime(time,type_ = "Y"):
    year = int(time.split('_')[1])
    try:
        quarter = int(time.split('_')[2])
        return year,quarter
    except:
        return year,0

def caculaterTime(time,period,type_= "Y"):
    year,quarter = formatTime(time,type_)
    if type_ == "Y":
        return year - period
    else:
        year = year + ((quarter - period-1)//4)
        quarter = (quarter - (period+1)%4 + 4 )%4+1
        return quarter,year

def get_data(list_str_all_code, dict_code, data, file_name):
    year,quater = caculaterTime(file_name,0,type_= "Q")
    year_p,quater_p = caculaterTime(file_name,1,type_= "Q")
    all_data_get = []
    maybemaso = ''
    maybe_get = []
    for col in data.columns:
        data_col = list(data[col])
        if find_Ma_so(data_col):
            maybemaso = col
            break
    for col in data.columns:
        data_col = list(data[col])        
        checkout, data_get = find_data_get(data_col)
        if checkout == True:
            if col == maybemaso:
                continue
            else:
                maybe_get.append(col)
                all_data_get.append(data_get)
    #xử lí feature
    # print(maybemaso, maybe_get)
    list_code = []
    list_feature = []
    list_data_get1 = all_data_get[0]
    try:
        list_data_get2 = all_data_get[1]
    except: 
        list_data_get2 = [np.nan]*len(list_data_get1)
    for value in list(data[maybemaso]):
        try:
            val = str(int(float(value)))
        except:
            val = str(value)

        if val in list_str_all_code:
            list_code.append(val)
            list_feature.append(dict_code[val])
        else:
            list_code.append(np.nan)
            list_feature.append(np.nan)
    df_return = pd.DataFrame({'code': list_code, 'feature': list_feature, f'{quater}/{year}': list_data_get1, f'{quater_p}/{year_p}': list_data_get2})
    return df_return  

def process_file_3(data_goc):
    #data_goc là dataframe đọc từ file scan, với header là None
    data = data_goc.copy()
    col1, col2 = 0, 0
    for id in range(len(data.columns)):
        data_col = list(data[data.columns[id]])
        if find_Ma_so(data_col):
            col1 = id
            col2 = id+1
            # print('check', maybemaso)
            break
    list1 = list(data[data.columns[col1]])
    list2 = list(data[data.columns[col2]])
    for i in range(len(list1)-1):
        if str(list1[i+1]) == 'nan':
            if '.' not in str(list1[i+1]).split(' ')[0]:
                list1[i+1] = list1[i]
    data[data.columns[col1]] = list1
    return data

def process_file_1_col(data_goc):
    #data_goc là dataframe đọc từ file scan, với header là None
    data = data_goc.copy()
    all_data = list(data[data.columns[0]])
    mile = 0
    for i in range(len(all_data)):
        item_i = str(all_data[i]).replace('(', '').replace(')', '').replace('.', '')
        if item_i.isdigit():
            mile = i
            print('mile', i)
            break
    all_data = all_data[mile:]
    process_all_data = []
    for i in range(len(all_data)):
        item_i = str(all_data[i]).replace('(', '').replace(')', '').replace('.', '')
        if item_i[:2].isdigit():
            process_all_data.append(all_data[i])

    mile_div = len(process_all_data)/3 
    print(mile_div)
    if mile_div % 1 != 0:
        return data_goc
    else:
        mile_div = int(mile_div)
        list1 = process_all_data[:mile_div]   
        list2 = process_all_data[mile_div:2*mile_div]
        list_code = []
        list_field = []
        for item in process_all_data[2*mile_div:]:
            list_split = str(item).split(' ')
            if list_split[0].isdigit():
                list_code.append(list_split[0])
                list_field.append(list_split[1:])
            else:
                list_code.append('nan')
                list_field.append(item)
            for i in range(len(list_code)-1):
                if str(list_code[i+1]) == 'nan':
                    if '.' not in list_field[0]:
                        list1[i+1] = list1[i]
        data_return = pd.DataFrame({'code':list_code, 'field':list_field, 'cuoinam':list2, 'daunam':list1})
        return data_return

(4, 2017)


In [120]:
def process(file_i):
    file_test = pd.read_excel(f'{PATH_FROM}{file_i}', header= None).dropna(axis=0, how='all')
    # file_test
    for col in file_test.columns:
        file_test.rename(columns = {col: str(col)}, inplace = True)
    # file_test1 = process_file_3(file_test)
    # file_test1
    try:
        # print(file_test)
        # print('vào 1')
        file_test1 = process_file_3(file_test)
        file_convert = get_data(list_str_all_code, dict_code, file_test1, file_i)

        file_convert = file_convert.dropna(subset=['code'], how='any')
        file_name = file_i.replace('.xlsx', '').replace('.xlsm', '')
        file_convert.to_csv(f'{PATH_TO}{file_name}.csv', index=False)
        # print('DONE1')
    except:
        try:
            # print('vào 2')
            # print(file_test)
            file_test1 = proocess_file0(file_test)
            file_convert = get_data(list_str_all_code, dict_code, file_test1, file_i)
            file_convert = file_convert.dropna(subset=['code'], how='any')
            file_name = file_i.replace('.xlsx', '').replace('.xlsm', '')
            file_convert.to_csv(f'{PATH_TO}{file_name}.csv', index=False)
            # print('DONE2')
        except:
            try:
                # print('vào 3')
                file_test1 = file_test.copy()
                file_convert = get_data(list_str_all_code, dict_code, file_test1, file_i)
                file_convert = file_convert.dropna(subset=['code'], how='any')
                file_name = file_i.replace('.xlsx', '').replace('.xlsm', '')
                file_convert.to_csv(f'{PATH_TO}{file_name}.csv', index=False)
                # print('DONE3')
            except:
                try:
                    # print('vào 4')
                    file_test1 = process_file(file_test)
                    file_convert = get_data(list_str_all_code, dict_code, file_test1, file_i)
                    file_convert = file_convert.dropna(subset=['code'], how='any')
                    file_name = file_i.replace('.xlsx', '').replace('.xlsm', '')
                    file_convert.to_csv(f'{PATH_TO}{file_name}.csv', index=False)
                    # print('DONE4')
                except:
                    try:
                        # print('vào 5')
                        file_test1 = process_file_1_col(file_test)
                        file_convert = get_data(list_str_all_code, dict_code, file_test1, file_i)
                        file_convert = file_convert.dropna(subset=['code'], how='any')
                        file_name = file_i.replace('.xlsx', '').replace('.xlsm', '')
                        file_convert.to_csv(f'{PATH_TO}{file_name}.csv', index=False)
                        # print('DONE5')
                    except:
                        list_file_toang.append(file_i)

In [126]:
F_FROM = "F0"
F_TO = "F1"
TYPE = "Quarter"
PATH_FROM = dict_path[F_FROM][TYPE]
PATH_TO = dict_path[F_TO][TYPE]
list_file_toang = []

In [ ]:
import os
list_name_file = os.listdir(PATH_FROM)
for file_i in list_name_file:
    print(file_i)
    process(file_i)

In [129]:
print(len(list_name_file),len(list_file_toang))

1159 235


# F2